# VacationPy
----

#### Note
* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key

pd.set_option('mode.chained_assignment', None)

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
weather_df = pd.read_csv("../WeatherPy/output_data/cities.csv")

### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [3]:
# Access maps with unique API key
gmaps.configure(api_key=g_key)

In [4]:
figure_layout = {
    'width': '400px',
    'height': '300px',
    'border': '1px solid black',
    'padding': '1px',
    'margin': '0 auto 0 auto'
}
fig = gmaps.figure(layout=figure_layout)
heatmap_layer = gmaps.heatmap_layer(
    weather_df[['Lat', 'Lng']], weights=weather_df['Humidity'],
    max_intensity=100, point_radius=5.0
)
fig.add_layer(heatmap_layer)
fig

Figure(layout=FigureLayout(border='1px solid black', height='300px', margin='0 auto 0 auto', padding='1px', wi…

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [5]:
ideal_weather_df = weather_df[(weather_df['Max Temp'] >= 70) & 
                              (weather_df['Max Temp'] <= 80) & 
                              (weather_df['Wind Speed'] <= 10) &
                              (weather_df['Cloudiness'] == 0) &
                              (weather_df['Humidity'] <= 50)]
ideal_weather_df

,Unnamed: 0,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed
246,274,pokhara,0.0,NP,1.584503e+09,41.0,28.23,83.98,71.47,0.83
248,276,batemans bay,0.0,AU,1.584503e+09,39.0,-35.72,150.18,75.99,1.99
267,299,etah,0.0,IN,1.584503e+09,46.0,27.63,78.67,71.13,5.97
307,340,baikunthpur,0.0,IN,1.584503e+09,35.0,23.25,82.55,74.01,0.85
399,439,babai,0.0,IN,1.584503e+09,22.0,22.70,77.93,79.72,1.03
423,466,bilma,0.0,NE,1.584503e+09,10.0,18.69,12.92,70.11,8.79
427,470,jati,0.0,PK,1.584503e+09,18.0,24.35,68.27,79.66,5.97


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [6]:
hotel_df = ideal_weather_df
hotel_df['Hotel Name'] = ""

In [7]:
base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
params = {
    "types": "lodging",
    "rankby": "distance",
    "key": g_key
}
# use iterrows to iterate through pandas dataframe
for index, row in hotel_df.iterrows():

    # get restaurant type from df
    location = f"{str(row['Lat'])}, {str(row['Lng'])}"

    # add keyword to params dict
    params['location'] = location

    # assemble url and make API request
    print(f"Retrieving Results for Index {index}: {location}.")
    response = requests.get(base_url, params=params).json()
    
    # extract results
    results = response['results']
    
    try:
        print(f"Closest hotel in {location} is {results[0]['name']}.")
        
        hotel_df.loc[index, 'Hotel Name'] = results[0]['name']
        
    except (KeyError, IndexError):
        print("Missing field/result... skipping.")
        
    print("------------")

Retrieving Results for Index 246: 28.23, 83.98.
Closest hotel in 28.23, 83.98 is Thakali bhancha and lodge.
------------
Retrieving Results for Index 248: -35.72, 150.18.
Closest hotel in -35.72, 150.18 is Shady Willows Caravan Park.
------------
Retrieving Results for Index 267: 27.63, 78.67.
Closest hotel in 27.63, 78.67 is Bhole BABA mandir.
------------
Retrieving Results for Index 307: 23.25, 82.55.
Closest hotel in 23.25, 82.55 is Vivek Jaiswal's Home.
------------
Retrieving Results for Index 399: 22.7, 77.93.
Closest hotel in 22.7, 77.93 is Neelesh Meena.
------------
Retrieving Results for Index 423: 18.69, 12.92.
Closest hotel in 18.69, 12.92 is Expeditions Ténére voyages.
------------
Retrieving Results for Index 427: 24.35, 68.27.
Closest hotel in 24.35, 68.27 is Muhammad Khuzaima Traders.
------------


In [8]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in ideal_weather_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [9]:
# Add marker layer ontop of heat map
markers = gmaps.marker_layer(locations, info_box_content=hotel_info)

fig.add_layer(markers)

# Display Map
fig

Figure(layout=FigureLayout(border='1px solid black', height='300px', margin='0 auto 0 auto', padding='1px', wi…